# Metrics
Diese Datei enthält Metriken und Hilfsfunktionen. Für Testzwecke der Hilfsfunktionen wird pymongo importiert und eine Verbindung zu einer MongoDB Instanz hergestellt.


In [1]:
import pymongo
client = pymongo.MongoClient('localhost', 27017)
raw = client.compas.compas

### Metrik analyseFrequency(db, pa)
Die Metrik analyseFrequency berechnet für alle Kategorien eines geschützten Attributs die Häufigkeit im Datensatz. Folgende Parameter werden benötigt:

    db: Die MongoDB collection in der der Datensatz liegt
    pa: Der Name des geschützten Attributs

Die Rückgabe der Funktion erfolgt in dem Format einer Liste von Tupeln. Jeder Tupel enthält die Information über den Namen der Kategorie und den Anteil der Kategorie im Datensatz.  
Beispiel einer Rückgabe:

    [('woman', 24000), ('man', 40000)]

In [2]:
def analyseFrequency(db, pa):
    categories = db.distinct(pa)
    return [
            (category, db.count_documents({pa : category})) 
                for category in categories
            ]

In [3]:
analyseFrequency(raw, 'Sex_Code_Text')

[('Female', 13329), ('Male', 47514)]

### Metrik analysePercentage(db, pa)
Die Metrik analysePercentage berechnet die Verteilung der Kategorien eines geschützten Attributs in einem Datensatz:

    db: Die MongoDB collection in der der Datensatz liegt
    pa: Der Name des geschützten Attributs

Die Rückgabe der Funktion erfolgt in dem Format einer Liste von Tupeln. Jeder Tupel enthält die Information über den Namen der Kategorie und deren Verteilung im Datensatz.  
Beispiel einer Rückgabe:

    [('woman', 0.4), ('man', 0.6)]

In [10]:
def analysePercentage(db, pa):
    frequencies = analyseFrequency(db, pa)
    length = db.count_documents({})
    return [(name, freq / length) for name, freq in frequencies]

In [11]:
analysePercentage(raw, 'Sex_Code_Text')

[('Female', 0.21907203786795523), ('Male', 0.7809279621320447)]

### Hilfsfunktion amountDownsample(frequencies)
Die Hilfsfunktion amountDownsample berechnet für alle Kategorien eines geschützten Attributs um wie viele Einträge sie reduziert werden soll. Als Input bekommt die Funktion eine Liste von Tupeln, wie diese die in analyseFrequency zurückgegeben wird.

    frequencies: Tupel aus Kategoriename des geschützten Attributs und der Häufigkeit im Datensatz

Die Rückgabe der Funktion erfolgt in dem Format einer Liste von Tupeln. Jeder Tupel enthält die Information über den Namen der Kategorie und den Anteil um den es reduziert werden soll.  
Beispiel einer Rückgabe:

    [('man', 16000)]

In [6]:
def amountDownsample(frequencies):
    if (len(frequencies) < 2):
        return []
    lowest_frequency_name, lowest_frequency = frequencies[0]
    for name, f in frequencies:
        if f < lowest_frequency:
            lowest_frequency_name, lowest_frequency = name, f
    if all(lowest_frequency == freq for _, freq in frequencies):
        return []
    return [
            (name, frequency - lowest_frequency) 
                for name, frequency in frequencies 
                if name != lowest_frequency_name
           ]

In [7]:
amountDownsample(analyseFrequency(raw, 'Sex_Code_Text'))

[('Male', 34185)]

### Hilfsfunktion amountUpsample(frequencies)
Die Hilfsfunktion amountUpsample berechnet im Gegensatz zu amountDownsample für alle Kategorien eines geschützten Attributs um wie viele Einträge sie erweitert werden soll. Als Input bekommt die Funktion ebenfalls eine Liste von Tupeln, wie diese die in analyseFrequency zurückgegeben wird.

    frequencies: Tupel aus Kategoriename des geschützten Attributs und der Häufigkeit im Datensatz

Die Rückgabe der Funktion erfolgt, analog zu amountDownsample, in dem Format einer Liste von Tupeln. Jeder Tupel enthält die Information über den Namen der Kategorie und den Anteil um den es reduziert werden soll.
Beispiel einer Rückgabe:

    [('woman', 16000)]

In [8]:
def amountUpsample(frequencies):
    if (len(frequencies) < 2):
        return []
    greatest_frequency_name, greatest_frequency = frequencies[0]
    for name, f in frequencies:
        if f > greatest_frequency:
            greatest_frequency_name, greatest_frequency = name, f
    if all(greatest_frequency == freq for _, freq in frequencies):
        return []
    return [
            (name, greatest_frequency - frequency) 
                for name, frequency in frequencies 
                if name != greatest_frequency_name
           ]

In [9]:
amountUpsample(analyseFrequency(raw, 'Sex_Code_Text'))

[('Female', 34185)]